In [1]:
import time
import pickle
import pandas as pd
import autosklearn.regression
from sklearn.metrics import mean_squared_error
pd.pandas.set_option('display.max_columns',None)
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("Number of latex.csv")
df = df[['Humidity', 'Wind', 'Tempature', 'Total(latex)(dry Milk)']]
df = df.dropna()

y = df['Total(latex)(dry Milk)']

# Select the features (all columns except the target variable)
X = df.drop('Total(latex)(dry Milk)', axis=1)
 

In [3]:
# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
print(X_train.shape)
print(y_train.shape)

(87, 3)
(87,)


In [5]:
start_time = time.time()

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=60,
    per_run_time_limit=30,
    tmp_folder="/tmp/autosklearn_regression",
    n_jobs = 5,
    memory_limit = 2500
)
automl.fit(X_train, y_train, dataset_name="latex_percision")

print("--- %s seconds ---" % (time.time() - start_time))

[WARNING] [2023-09-09 17:03:38,419:Client-AutoML(1):latex_percision] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
--- 59.249265909194946 seconds ---


In [6]:
print(automl.leaderboard())

          rank  ensemble_weight              type      cost  duration
model_id                                                             
29           1             0.50    ard_regression  0.581215  0.714862
24           2             0.06       extra_trees  0.616360  1.422728
36           3             0.20          adaboost  0.630310  0.767588
16           4             0.18       extra_trees  0.800124  1.436451
26           5             0.02  gaussian_process  0.889262  0.980988
32           6             0.04     random_forest  0.916191  1.216803


In [7]:
# with open('ccl_regression_model.pickle', 'rb') as file:
#     automl = pickle.load(file)

In [11]:
predictions = automl.predict(X_test)
mse_value = mean_squared_error(y_test, predictions)
print(mse_value)

83.07264263475332


In [10]:
with open('latex_model.pickle', 'wb') as file:
    pickle.dump(automl, file, protocol=pickle.HIGHEST_PROTOCOL)